# LookML for Refinement Layers


Stage One - Write the body code for each file.
Stage Two - Generate a full script for each refinement layer as well as the explore.
Stage Three - Generate the files with file names.
Stage Four - Fully automate the process of creating files in Github.


In [6]:
# Name this model
# i.e. datagrid.marketing_dm.impact_partner_performance_report
view_name = "marketing_aop_actuals"
print(view_name)

marketing_aop_actuals


In [1]:
# Paste in the dimensions from your raw lookML code.
text = """

  dimension: ad_week {
    description: ""
    type: date_week
    hidden: yes
  }
  dimension: spend {
    group_label: "Spend"
    description: ""
    value_format: "$#,##0"
    type: number
    hidden: yes
  }
  dimension: search_spend {
    group_label: "Spend"
    description: ""
    value_format: "$#,##0"
    type: number
    hidden: yes
  }
  dimension: social_spend {
    group_label: "Spend"
    description: ""
    value_format: "$#,##0"
    type: number
    hidden: yes
  }
  dimension: marketplace_spend {
    group_label: "Spend"
    description: ""
    value_format: "$#,##0"
    type: number
    hidden: yes
  }


"""

print(text)



  dimension: ad_week {
    description: ""
    type: date_week
    hidden: yes
  }
  dimension: spend {
    group_label: "Spend"
    description: ""
    value_format: "$#,##0"
    type: number
    hidden: yes
  }
  dimension: search_spend {
    group_label: "Spend"
    description: ""
    value_format: "$#,##0"
    type: number
    hidden: yes
  }
  dimension: social_spend {
    group_label: "Spend"
    description: ""
    value_format: "$#,##0"
    type: number
    hidden: yes
  }
  dimension: marketplace_spend {
    group_label: "Spend"
    description: ""
    value_format: "$#,##0"
    type: number
    hidden: yes
  }





In [3]:
def parse_lookml(lookml_code):
    dimensions_string = []
    dimensions_number = []
    dimensions_time = []
    dimensions_boolean = []

    lines = lookml_code.split('\n')
    for line in lines:
        if line.strip().startswith('dimension'):
            dimension_name = line.split(':')[1].strip().split('{')[0].strip()  # Remove " {" at the end
            type_line = next(l for l in lines[lines.index(line):] if 'type:' in l)
            dimension_type = type_line.split(':')[1].strip()
            if dimension_type == 'string':
                dimensions_string.append(dimension_name)
            elif '_id' in dimension_name:
                dimensions_string.append(dimension_name)
            elif dimension_type == 'number':
                dimensions_number.append(dimension_name)
            elif dimension_type == 'time':
                dimensions_time.append(dimension_name)
            elif dimension_type == 'yesno':
                dimensions_boolean.append(dimension_name)

    return dimensions_string, dimensions_number, dimensions_time, dimensions_boolean


strings, numbers, times, booleans = parse_lookml(text)

print("String dimensions:", len(strings))
print("Boolean dimensions:", len(booleans))
print("Number dimensions:", len(numbers))
print("Time dimensions:", len(times))


String dimensions: 0
Boolean dimensions: 0
Number dimensions: 4
Time dimensions: 0


In [ ]:
strings

In [ ]:
booleans

In [4]:
numbers

['spend', 'search_spend', 'social_spend', 'marketplace_spend']

In [ ]:
times

In [7]:
# create functional layer filename
print(view_name + '.functional')


marketing_aop_actuals.functional


In [8]:
# create functional view file

def format_string(view_name):
    return f'''include: "/1_raw/{view_name}.view"

view: +{view_name} {{

}}'''

formatted_string = format_string(view_name)
print(formatted_string)

include: "/1_raw/marketing_aop_actuals.view"

view: +marketing_aop_actuals {

}


In [ ]:
## Set Primary Key

def set_primary_key(list1, list2):
    looml = "## PRIMARY KEY\n\n"
    for item in list1 + list2:            
            if "sk" in item.lower():
                looml += f"dimension: {item} {{\n"
                looml += "    primary_key: yes\n"
                looml += "}\n"
                looml += "\n"
    return looml

# Example lists

result = set_primary_key(strings, numbers)
print(result)



In [9]:
# create metrics
def generate_measure_lookml(names):
    lookml_code = '## BASIC METRICS\n\n'
    for name in names:
        lookml_code += f'measure: total_{name} {{\n'
        lookml_code += '  type: sum\n'
        lookml_code += f'  sql: ${{{name}}} ;;\n'
        lookml_code += '}\n\n'
    return lookml_code

generated_lookml = generate_measure_lookml(numbers)
print(generated_lookml)

## BASIC METRICS

measure: total_spend {
  type: sum
  sql: ${spend} ;;
}

measure: total_search_spend {
  type: sum
  sql: ${search_spend} ;;
}

measure: total_social_spend {
  type: sum
  sql: ${social_spend} ;;
}

measure: total_marketplace_spend {
  type: sum
  sql: ${marketplace_spend} ;;
}




In [ ]:
# create logical layer filename
print(view_name + '.logic')

In [ ]:
# create logical view file

def format_string(view_name):
    return f'''include: "/2_functional/{view_name}.functional.view"

view: +{view_name} {{

}}'''

formatted_string = format_string(view_name)
print(formatted_string)

In [ ]:
# create visual layer filename
print(view_name + '.visual')

In [ ]:
# create visual view file

def format_string(view_name):
    return f'''include: "/3_logic_layer/{view_name}.logic.view"

view: +{view_name} {{

}}'''

formatted_string = format_string(view_name)
print(formatted_string)

In [ ]:
## HIDE Primary Key & IDs

def hide_fields(list1, list2):
    looml = "\n\n## PRIMARY KEY & IDs \n\n"
    for item in list1 + list2:
        if "_id" in item.lower() or "sk" in item.lower() or "related_tranid" in item.lower():
            looml += f"dimension: {item} {{\n    hidden: yes\n"
            looml += "}\n"
            looml += "\n"
    return looml

# Example lists

result = hide_fields(strings, numbers)
print(result)



In [ ]:
## HIDE Dimensions for Metrics

def hide_fields(list1):
    looml = "\n\n## HIDDEN DIMENSIONS for METRICS \n\n"
    for item in list1:
            looml += f"dimension: {item} {{\n    hidden: yes\n"
            looml += "}\n"
            looml += "\n"
    return looml

# Example lists

result = hide_fields(numbers)
print(result)

In [ ]:
# METRICS FORMATING
def generate_measure_lookml(names):
    lookml_code = '\n\n## METRICS\n\n'
    conditions = ['spend', 'revenue', 'cost', 'sales', 'asp', 'discount', 'roas']
    for name in names:
        # Set label and value format based on conditions

        if any(condition in name.lower() for condition in conditions):
            label = name.replace('_', ' ').title()  # Replace _ with space and make it proper case
            value_format = '"$#,##0"'
        else:
            label = name.replace('_', ' ').title()  # Replace _ with space and make it proper case
            value_format = '"#,##0"'

        lookml_code += f'measure: total_{name} {{\n'
        lookml_code += f'  label: "{label}"\n'
        lookml_code += f'  value_format: {value_format}\n'
        lookml_code += '}\n\n'
    return lookml_code

generated_lookml = generate_measure_lookml(numbers)
print(generated_lookml)

In [ ]:
## Date Formating

def generate_lookml(names):
    
    lookml_code = '\n\n## DATES & TIMES\n\n'
    
    for name in names:
        lookml_code += f'dimension_group: {name} {{\n'
        lookml_code += '  group_label: "Date"\n'
        lookml_code += '  label: ""\n'
        lookml_code += '}\n\n'
    return lookml_code

generated_lookml = generate_lookml(times)
print(generated_lookml)


In [ ]:
# create Explore file
print(view_name + '.explore')

In [ ]:
## Generate Explore lookML
lookML_code = ""
explore_label = ' '.join(word.capitalize() for word in view_name.split('_'))

lookML_code += f'include: "/3_visual/{view_name}.visual.view"\n\n'
lookML_code += f'explore: {view_name} {{\n'
lookML_code += f'   from: {view_name}\n'
lookML_code += f'   label: "{explore_label}"\n'
lookML_code += f'   group_label: "Marketing"\n'
lookML_code += f'}}'

print(lookML_code)

In [ ]:
## Commit Code
print(f'Deploy {explore_label} Explore into production.')